In [ ]:
import logging
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from neurolib.utils.loadData import Dataset
from neurolib.models.aln import ALNModel
import neurolib.optimize.exploration.explorationUtils as eu
from neurolib.utils.parameterSpace import ParameterSpace

from evaluation import param_search

logger = logging.getLogger()
warnings.filterwarnings("ignore")
logger.setLevel(logging.INFO)

results_path = "/Users/valery/Downloads/"

### SEE: https://github.com/neurolib-dev/neurolib/blob/master/neurolib/models/aln/loadDefaultParams.py

In [ ]:

ds = Dataset("gw")
model = ALNModel(Cmat=ds.Cmat, Dmat=ds.Dmat)
model.params['dt'] = 0.1
model.params['duration'] = 20 * 1000  # ms

# Sleep model from newest evolution October 2020
model.params["b"] = 3.2021806735984186
model.params["tauA"] = 4765.3385276559875
model.params["sigma_ou"] = 0.36802952978628106
model.params["Ke_gl"] = 265.48075753153
model.params["signalV"] = 80.0
# model.params["mue_ext_mean"] = 3.3202829454334535
model.params["mui_ext_mean"] = 3.682451894176651

# add custom parameter for downsampling results
# 10 ms sampling steps for saving data, should be multiple of dt
model.params['save_dt'] = 10.0


parameters = ParameterSpace(
    {# Reduction of GABA levels (GABA inhibitory neurotransmitter)
     # -> reduction of inhibitory weights
     # "mui_ext_mean": np.linspace(3.68, 1, 10), # ???
     # Increased excitability of excitatory neurons (compensating for NMDAR hypofunction at E population)
     # -> increased mu_E
     "mue_ext_mean": np.linspace(3.32, 5, 10),
     # NMDA receptor hypofunction (preferentially at inhibitory interneurons)
     # -> decreased E->I weight
     # Maximum synaptic current from E to I
     "Jie_max": np.linspace(2.6, 1, 10),
     # General structural dysconnectivity -> reduction of global coupling
     "Ke_gl": [200., 265.5] 
    })


search = param_search(model, parameters, fname=results_path + "scz_sleep.hdf")


In [ ]:
search.loadResults(filename=results_path + "scz_sleep.hdf", all=False)

In [ ]:
eu.plotExplorationResults(search.dfResults,
                          par1=['mue_ext_mean', 'Input to E [nA]'],
                          par2=['Jei_max', 'Synaptic current from E to I [nA]'],
                          by=["Ke_gl"],
                          plot_key='normalized_down_lengths_mean',
                          plot_clim=[0.0, 100.0],
                          nan_to_zero=False,
                          plot_key_label="Number of local waves (%)",
                          one_figure=True,
                          # multiply_axis=0.2,
                          # contour=["max_amp_output", "up_down_difference"],
                          # contour_color=[['white'], ['springgreen']],
                          # contour_levels=[[10], [10]],
                          # contour_alpha=[1.0, 1.0],
                          # contour_kwargs={0: {"linewidths": (5,)},
                          #                 1: {"linestyles": "--",
                          #                     "linewidths": (5,)}},
                          # # alpha_mask="relative_amplitude_BOLD",
                          # mask_threshold=0.1,
                          # mask_alpha=0.2,
                          savename=results_path + "scz_sleep.pdf")

In [ ]:
search.dfResults.columns